# Class 5 — Lists → Dictionaries & Sets (State • Transitions • Invariants)

**Course lens:** Every program has **state**; code performs **transitions**; correct programs preserve **invariants**.

This notebook is designed for a 90-minute class. It starts with **list indexing** as a bridge to **dictionary keys**, then covers **sets** as “invariant-enforcing collections” (uniqueness + membership).


## 0. Agenda (90 minutes)
1. Lists recap: **indexing** and the index invariant (0 … len-1)
2. The limitation of positional meaning (why parallel lists break)
3. Dictionaries: **keys** as “semantic indexes”
4. Conversions:
   - two lists → dict (`zip`)
   - dict → list of `(key, value)` tuples (`items`)
5. Sets: uniqueness + membership; set algebra
6. 15 in-class problems


## 1. Lists recap: indexing is a contract
A list is an **ordered** collection. Its elements live at integer positions called **indexes**.

**Index invariant (for a list `L`):**
- Valid indexes are integers in the range `0` to `len(L) - 1`.
- `len(L)` is **not** a valid index (it is “one past the end”).
- Negative indexes are allowed and count from the right (`-1` is the last item).

We’ll use this as a bridge: **dictionary keys play the role of indexes**, but they’re meaningful labels.


In [ ]:
# Observation vs transition (lists)
nums = [10, 20, 30, 40]
print('state nums =', nums)

# Observation (does not change the list)
print('nums[0] =', nums[0])
print('nums[len(nums)-1] =', nums[len(nums)-1])

# Transition (changes the list)
nums[1] = 99
print('after transition nums =', nums)


### 1.1 The index invariant in action
Predict what happens before you run each line.
- What is the largest valid index?
- What happens at `nums[len(nums)]`?
- What does `nums[-1]` mean?


In [ ]:
# Try these one at a time (comment/uncomment)
nums = [10, 20, 30, 40]
print('len(nums) =', len(nums))
print('largest valid index =', len(nums) - 1)

# print(nums[len(nums)])   # IndexError: out of range (violates index invariant)
print(nums[-1])            # last element
print(nums[-2])            # second-to-last


## 2. Why we need dictionaries: parallel lists are fragile
A common beginner pattern is to store related data in two lists:
- `names[i]` goes with `scores[i]`

**Invariant (parallel lists):** for every valid index `i`, `names[i]` and `scores[i]` refer to the same person.

This invariant is easy to break if we insert, delete, or reorder one list without doing the same to the other.


In [ ]:
names  = ['Bob', 'Alice', 'Jen']
scores = [  90,     85,    92]
print('names =', names)
print('scores=', scores)

# Observation: look up Bob's score using positional meaning
i = names.index('Bob')
print("Bob's score is", scores[i])

# Transition that breaks the invariant (reordering only one list)
scores.sort()  # sorts numbers, but names did not move with them
print('\nAfter scores.sort():')
print('names =', names)
print('scores=', scores)
print("Now Bob's score looks like", scores[names.index('Bob')], '(wrong!)')


## 3. Dictionaries: keys are semantic indexes
A dictionary maps **keys → values**.

**Dictionary invariant:**
- Each key is **unique**.
- Looking up by key retrieves the associated value.
- Keys are typically immutable types (e.g., strings, numbers, tuples of immutables).

In our lens:
- **State:** the current set of key→value associations
- **Transition:** insertion/update/deletion of a key
- **Observation:** lookup without changing the mapping


In [ ]:
scores = {'Bob': 90, 'Alice': 85, 'Jen': 92}
print('state scores =', scores)

# Observation
print("scores['Bob'] =", scores['Bob'])

# Transition: update
scores['Bob'] = 95
print('after update scores =', scores)

# Transition: insert
scores['Zena'] = 88
print('after insert scores =', scores)

# Transition: delete
del scores['Alice']
print('after delete scores =', scores)


### 3.1 Safe lookup (avoiding KeyError)
Two common patterns:
- `in` to test membership (observation)
- `.get(key, default)` to supply a fallback


In [ ]:
scores = {'Bob': 95, 'Jen': 92}
name = 'Alice'

print('Is', name, 'a key?', name in scores)  # observation
print('Lookup with get:', scores.get(name, 'no score recorded'))


## 4. Conversion: two lists → dictionary (keys list + values list)
If `keys` and `values` are parallel lists, we can build a dictionary with `zip`.

**Invariant required:** `len(keys) == len(values)` so every key gets a corresponding value.

### 4.1 Using `zip`
- `zip(keys, values)` produces pairs `(keys[i], values[i])`
- `dict(...)` consumes those pairs to build a dictionary


In [ ]:
keys   = ['Bob', 'Alice', 'Jen']
values = [90, 85, 92]

print('keys  =', keys)
print('values=', values)
print('invariant len(keys)==len(values)?', len(keys) == len(values))

d = dict(zip(keys, values))
print('dictionary d =', d)


### 4.2 What if lengths differ?
`zip` stops at the shorter list.
That means information is silently dropped — an invariant failure in your *data model*.


In [ ]:
keys   = ['Bob', 'Alice', 'Jen']
values = [90, 85]  # missing one score
d = dict(zip(keys, values))
print('d =', d)
print('Notice Jen is missing because zip stopped early.')


## 5. Conversion: dictionary → list of tuples `(key, value)`
Dictionaries provide `.items()` which behaves like an iterable of `(key, value)` pairs.

- `list(d.items())` makes a **new list** of **tuples**.
- Each tuple is a small, immutable record: `(key, value)`.

**Invariant:** each tuple has length 2, and the key is unique across tuples.


In [ ]:
d = {'Bob': 90, 'Alice': 85, 'Jen': 92}
pairs = list(d.items())
print('d =', d)
print('pairs =', pairs)
print('type(pairs) =', type(pairs))
print('type(pairs[0]) =', type(pairs[0]))


### 5.1 Sorting those pairs (observation vs transition)
- `sorted(pairs)` returns a new list (new state creation)
- `pairs.sort()` mutates the list (transition)


In [ ]:
d = {'Bob': 90, 'Alice': 85, 'Jen': 92}
pairs = list(d.items())

new_pairs = sorted(pairs)  # new state creation
print('pairs (original) =', pairs)
print('new_pairs        =', new_pairs)

pairs.sort()  # transition
print('pairs after sort =', pairs)


## 6. Iterating through dictionaries
Three common iteration views:
- `for k in d:` (keys)
- `for v in d.values():`
- `for (k, v) in d.items():`

**Invariant during iteration:** you should not add/remove keys while iterating over the dictionary.
(Updating existing values is usually fine; changing keys/size while iterating is risky.)


In [ ]:
d = {'Bob': 90, 'Alice': 85, 'Jen': 92}

print('Keys:')
for k in d:
    print(' ', k)

print('\nValues:')
for v in d.values():
    print(' ', v)

print('\nKey-value pairs:')
for k, v in d.items():
    print(f'  {k} -> {v}')


## 7. Sets: enforcing uniqueness + fast membership
A set is an **unordered** collection of **unique** elements.

**Set invariants:**
- No duplicates (uniqueness is enforced automatically)
- Membership matters (`x in s`)
- No indexing (because no order)

In our lens:
- **State:** which elements are currently members of the set
- **Transition:** `add`, `remove`, `discard`, `pop`, `clear`
- **Observation:** membership tests and set operations that return new sets


In [ ]:
letters = set('HELLO')
print('letters =', letters)
print('len(letters) =', len(letters), '(duplicates removed)')

# Observation
print("'H' in letters?", 'H' in letters)
print("'Z' in letters?", 'Z' in letters)

# Transition
letters.add('Z')
print('after add Z:', letters)

# discard does not error if missing
letters.discard('Q')
print('after discard Q (no error):', letters)


### 7.1 Set algebra (new state creation)
- Union: `A | B`
- Intersection: `A & B`
- Difference: `A - B`
- Symmetric difference: `A ^ B`

These return **new sets** (new state creation), leaving `A` and `B` unchanged.


In [ ]:
A = {1, 2, 3, 4}
B = {3, 4, 5}
print('A =', A)
print('B =', B)
print('A | B =', A | B)
print('A & B =', A & B)
print('A - B =', A - B)
print('A ^ B =', A ^ B)
print('A still =', A)
print('B still =', B)


## 8. Quick invariants by data type (your “always true unless a bug” list)
Use these as mental contracts while reading code.

### 8.1 `int`
- Whole numbers, arbitrary precision (in Python)
- Immutable (operations create new ints)

### 8.2 `float`
- Approximate real numbers (binary floating-point)
- Rounding error is normal (invariant: *approximate*, not exact)
- Immutable

### 8.3 `bool`
- Exactly two values: `True` and `False`
- Logical operators preserve truth tables (invariants of `and`, `or`, `not`)
- Immutable

### 8.4 `str`
- Sequence of characters
- Indexed from `0` … `len(s)-1` (same index invariant as lists)
- Immutable (methods return new strings)

### 8.5 `list`
- Ordered sequence
- Index invariant: valid indexes are `0` … `len(L)-1`
- Mutable (methods like `append`, `sort` mutate)

### 8.6 `tuple`
- Ordered sequence
- Index invariant: valid indexes are `0` … `len(t)-1`
- Immutable (good for fixed records)

### 8.7 `dict`
- Mapping from unique keys to values
- Keys are unique; lookup by key retrieves value
- Keys must be hashable (typically immutable)
- Mutable (adding/removing keys changes state)

### 8.8 `set`
- Unordered collection of unique elements
- Membership-focused; no indexing
- Mutable (`add`, `remove`), but set algebra operations can create new sets


## 9. Summary
- Lists use **integer indexes** (`0 … len-1`) — great for ordered sequences.
- Parallel lists require a fragile invariant (“these two lists stay aligned”).
- Dictionaries replace positional meaning with **semantic keys**.
- `dict(zip(keys, values))` converts two aligned lists into a dictionary.
- `list(d.items())` converts a dictionary into a list of `(key, value)` tuples.
- Sets enforce the invariant of **uniqueness** and support fast membership and algebra.


## 10. In-class problems (15)
**Directions for each problem:**
1. Write your prediction (as a comment).
2. Run the code.
3. Explain the result using **state, transitions, invariants**.


### Problem 1 — List index invariant


In [ ]:
nums = [5, 6, 7]
# Predict: largest valid index?
print("len(nums) =", len(nums))
print("largest valid index =", len(nums) - 1)
# Predict: what happens here?
# print(nums[len(nums)])

### Problem 2 — Negative indexes


In [ ]:
s = "PYTHON"
# Predict each:
print(s[-1])
print(s[-2])
print(s[0], s[1])

### Problem 3 — Parallel lists invariant


In [ ]:
names  = ["A", "B", "C"]
scores = [10, 20, 30]
# Transition that breaks invariant:
scores.reverse()
# Observation: what score does B appear to have now?
i = names.index("B")
print(scores[i])

### Problem 4 — Build dict with zip


In [ ]:
keys = ["A", "B", "C"]
vals = [1, 2, 3]
d = dict(zip(keys, vals))
print(d)
# Observation: lookup
print(d["B"])

### Problem 5 — Zip length mismatch


In [ ]:
keys = ["A", "B", "C"]
vals = [1, 2]
d = dict(zip(keys, vals))
print(d)
# Which key is missing and why (invariant failure)?

### Problem 6 — Updating a dict value


In [ ]:
d = {"x": 1, "y": 2}
# Transition:
d["x"] = d["x"] + 10
print(d)

### Problem 7 — Insert vs update


In [ ]:
d = {"x": 1}
d["x"] = 99   # update
d["y"] = 100  # insert
print(d)

### Problem 8 — Safe lookup with get


In [ ]:
d = {"x": 1}
print(d.get("x", 0))
print(d.get("z", 0))

### Problem 9 — Items to list of tuples


In [ ]:
d = {"b": 2, "a": 1, "c": 3}
pairs = list(d.items())
print(pairs)

### Problem 10 — Sorted pairs (new state creation)


In [ ]:
d = {"b": 2, "a": 1, "c": 3}
pairs = list(d.items())
new_pairs = sorted(pairs)
print("pairs     =", pairs)
print("new_pairs =", new_pairs)

### Problem 11 — Set removes duplicates


In [ ]:
letters = set("MISSISSIPPI")
print(letters)
print("len =", len(letters))

### Problem 12 — Membership as observation


In [ ]:
s = {"cat", "dog", "fish"}
print("dog" in s)
print("cow" in s)

### Problem 13 — Set add is idempotent


In [ ]:
s = {1, 2, 3}
s.add(2)
s.add(4)
print(s)

### Problem 14 — Set algebra creates new state


In [ ]:
A = {1, 2, 3}
B = {3, 4}
C = A | B
print("A =", A)
print("B =", B)
print("C =", C)

### Problem 15 — Dict keys must be hashable


In [ ]:
# Predict: will this work?
# d = { [1,2,3] : "list as key" }
# What invariant is being enforced about keys?